## 01. 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import regex as re
import gc
import matplotlib.pyplot as plt
import pyarrow.feather as feather
import seaborn as sns
import datatable as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


## 02. 데이터 불러오기

In [2]:
preData = pd.read_parquet('/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet')
preLabel = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv', low_memory=True)

## 03. 데이터 준비

In [3]:
# 데이터 정제 : 50% 이상 missing data 날려주기
def process_data(data):
    missing_data = [col for col in data.columns if data[col].isna().sum() >  0.5*len(data.index)]
    data = data.drop(columns=missing_data)
    
    # Feature engineering: number of transactions
    numTx = data['customer_ID'].value_counts().tolist()
    data = data.groupby('customer_ID').tail(1)
    data = data.reset_index(drop=True)
    data.insert(2,"numTx",numTx)
    data = data.drop(columns=["S_2", "customer_ID"])

    # 데이터 채우기
    for i in data.columns:
        data[i] = data[i].fillna(data[i].mean())
    return data    

In [4]:
preData = process_data(preData)

## 04. Train Test Split

In [5]:
X_train, X_val, y_train, y_val = train_test_split(preData,preLabel['target'],test_size=0.2, random_state=42)

In [6]:
del preData, preLabel
gc.collect()

42

## 05. CATBOOST 모델 쌓기

In [7]:
import catboost as cat
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

clf = CatBoostClassifier(iterations=5000, task_type="GPU",random_state=22, bagging_temperature = 0.2)
clf.fit(X_train,y_train,eval_set=(X_val,y_val),verbose=True)

Learning rate set to 0.022138
0:	learn: 0.6652790	test: 0.6652138	best: 0.6652138 (0)	total: 21.5ms	remaining: 1m 47s
1:	learn: 0.6411052	test: 0.6410376	best: 0.6410376 (1)	total: 45.1ms	remaining: 1m 52s
2:	learn: 0.6176401	test: 0.6174890	best: 0.6174890 (2)	total: 60.2ms	remaining: 1m 40s
3:	learn: 0.5953343	test: 0.5951251	best: 0.5951251 (3)	total: 75.5ms	remaining: 1m 34s
4:	learn: 0.5733362	test: 0.5731131	best: 0.5731131 (4)	total: 90ms	remaining: 1m 29s
5:	learn: 0.5524560	test: 0.5522358	best: 0.5522358 (5)	total: 104ms	remaining: 1m 26s
6:	learn: 0.5338796	test: 0.5336204	best: 0.5336204 (6)	total: 118ms	remaining: 1m 24s
7:	learn: 0.5186591	test: 0.5183633	best: 0.5183633 (7)	total: 132ms	remaining: 1m 22s
8:	learn: 0.5040833	test: 0.5037528	best: 0.5037528 (8)	total: 146ms	remaining: 1m 21s
9:	learn: 0.4901297	test: 0.4897619	best: 0.4897619 (9)	total: 160ms	remaining: 1m 20s
10:	learn: 0.4756183	test: 0.4752694	best: 0.4752694 (10)	total: 180ms	remaining: 1m 21s
11:	lear

In [8]:
prediction = clf.predict_proba(X_val)
rounded_predictions = np.argmax(prediction, axis=-1)
c_matrix = confusion_matrix(y_val,rounded_predictions)
dt_acc = c_matrix.trace()/c_matrix.sum()
print(c_matrix)
print(dt_acc)

[[63813  4427]
 [ 4615 18928]]
0.9014850244598673


In [9]:
#cleanup
del X_train, X_val, y_train, y_val
gc.collect()

42

## 06. Test Data 준비

In [10]:
testData = pd.read_parquet('/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet')
testCustomer  = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv', usecols=['customer_ID'], low_memory=True)

In [11]:
testData = process_data(testData)

In [12]:
prediction = clf.predict_proba(testData)
final_predictions = prediction[:,1]

In [13]:
output = pd.DataFrame({'customer_ID': testCustomer.customer_ID, 'prediction': final_predictions})
output.to_csv('submission.csv', index=False)

## 제출 score : 0.786